In [16]:
%pylab qt

import os
import itertools
import cv2
import numpy as np
from glob import glob
from sklearn.neighbors import NearestNeighbors

import helpers.classes as ch
import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(ch)
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline/'

def register_outline_with_icp(reference, points, no_iterations = 3):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(reference)

    for i in range(no_iterations):
        #Find the nearest neighbours between the current source and the
        #destination cloudpoint
        distances, indices = nbrs.kneighbors(points)
        indices = np.array(indices).flatten()
        
        #Compute the transformation between the current source
        #and destination cloudpoint
        R, t = gh.estimate_rigid_transform(points, reference[indices])
        R = np.array(R)
        t = np.array(t)
        #Transform the previous source and update the
        #current source cloudpoint
        points = (np.dot(R, points.transpose())).transpose() + np.tile(t, (points.shape[0], 1))
    
    return points

def do_registration(loaded):
    transposed_and_scaled = []
    registered = []
           
    for outline in loaded:
        points = outline['points']
        
        centroid = mean(points, axis=0)
        points = points - np.tile(centroid, (len(points), 1))
        
        scale = sqrt(np.sum(np.power(points, 2)) / len(points))
        points = np.divide(points, scale)
        points[: ,0] = -points[:, 0]
        
        edges = outline['edges']
        points, edges = gh.normalize_outline(points, outline['edges'])
        
        transposed_and_scaled.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    dh.outlines(transposed_and_scaled, show_direction=True)
    reference = max(transposed_and_scaled, key=lambda o: o['points'].shape[0])['points']
    for outline in transposed_and_scaled:
        points = outline['points']
        edges = outline['edges']
        
        if not np.array_equal(points, reference):
            points = register_outline_with_icp(reference, points)
            points, edges = gh.normalize_outline(points, outline['edges'])
        registered.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    return registered

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


In [10]:
filenames = glob(DATA_PATH + '*.npz')
loaded = []
for f in filenames:
    basename = os.path.basename(f)
    tri = np.load(f)
    
    points = tri['points']
    triangles = tri['triangles']
        
    label = ''.join([i if ord(i) < 128 else ' ' for i in basename])
    
    cls = ch.get_class(basename)
        
    outline_points, outline_edges = gh.extract_outline(points, triangles)
    
    loaded.append({
        'label': label,
        'filename': f,
        'class': cls[0],
        'class_label': cls[1],
        'points': outline_points,
        'edges': outline_edges
    })

In [11]:
filtered = ch.filter_by_classes(loaded, [ 2, 3 ])

In [17]:
registered = do_registration(filtered)

In [13]:
dh.outlines(registered, show_direction=True)

In [3]:
for outline in registered:
    basename = os.path.basename(outline['filename'])
    destination = os.path.join(TO_PATH, basename)
    
    np.savez(destination, points=outline['points'], edges=outline['edges'])